# Imports

In [103]:
#imports 
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Perceptron
from sklearn.metrics import confusion_matrix


#print('imports finished')

# Data Input and Preprocessing

# Perceptron Model

In [146]:
#create the model
per_model = Perceptron(max_iter=1000, tol=1e-3, random_state=42)
#fit the model
per_model.fit(X_train, y_train)

#make predictions
y_pred_perceptrons = per_model.predict(X_test)


# Compute the confusion matrix
cm = confusion_matrix(y_test, y_pred_perceptrons)

# Print the confusion matrix
print(cm)


# summarize result

[[69  2]
 [ 1 42]]


# Sequential Deep Neural Network

Build, train, and compile model

In [149]:
model = keras.Sequential([
      layers.Dense(64, activation='relu'),

      layers.Dense(64, activation='relu'),

      layers.Dense(64, activation = 'sigmoid'),

      layers.Dense(1, activation ='sigmoid')

  ])
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001))

model.fit(X_train,y_train, epochs = 100, verbose = 1)


Epoch 1/100
15/15 [==============================] - 1s 3ms/step - loss: 0.9754
Epoch 2/100
15/15 [==============================] - 0s 5ms/step - loss: 0.4671
Epoch 3/100
15/15 [==============================] - 0s 4ms/step - loss: 0.2469
Epoch 4/100
15/15 [==============================] - 0s 5ms/step - loss: 0.1649
Epoch 5/100
15/15 [==============================] - 0s 5ms/step - loss: 0.1235
Epoch 6/100
15/15 [==============================] - 0s 4ms/step - loss: 0.0993
Epoch 7/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0860
Epoch 8/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0757
Epoch 9/100
15/15 [==============================] - 0s 4ms/step - loss: 0.0679
Epoch 10/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0619
Epoch 11/100
15/15 [==============================] - 0s 5ms/step - loss: 0.0565
Epoch 12/100
15/15 [==============================] - 0s 4ms/step - loss: 0.0520
Epoch 13/100
15/15 [=================

Make predictions and assess accuracy

In [147]:
y_pred_dnn = model.predict(X_test)



for prediction in y_pred_dnn:
  print(prediction, end = ' ')





4/4 [==============================] - 0s 3ms/step
[0.00134107] [0.9996368] [0.9996323] [0.00047132] [0.00047097] [0.99963695] [0.99963695] [0.9995145] [0.02443114] [0.00047172] [0.02327011] [0.9995808] [0.0004981] [0.9994374] [0.0004718] [0.9996358] [0.00047314] [0.00047117] [0.0004708] [0.99963695] [0.00074721] [0.00049216] [0.99963695] [0.00047259] [0.0004719] [0.00047097] [0.00047134] [0.0004845] [0.00048224] [0.99963695] [0.00047109] [0.00047111] [0.00047778] [0.00051081] [0.0004714] [0.00049076] [0.9994908] [0.00048953] [0.9996368] [0.00062979] [0.00047113] [0.99963266] [0.00047301] [0.00047811] [0.00083321] [0.09656901] [0.00047088] [0.00047589] [0.00105856] [0.00047265] [0.9996368] [0.99963695] [0.00887937] [0.00058394] [0.00047081] [0.00050862] [0.00047109] [0.99963695] [0.9995957] [0.00047097] [0.00049013] [0.99963695] [0.99963695] [0.00056409] [0.00048081] [0.00109015] [0.99963695] [0.99963695] [0.00047379] [0.00048293] [0.9996287] [0.9996336] [0.00047475] [0.9996318] [0.000